In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [3]:
#attempt to make a better/more streamlined form for reading the header and data

#important steps:
# 1 - open file
# 2 - split by section (or simple iterate by line and change approach at "#" symbol)
# 3 - split header section into lines then fields and store (or directly to fields (i.e. from one single string))
# 4 - split intensity section into lines then into values and store all in a single list
# 5 - make numpy array from the list of intensity values and reshape to dimensions given in header information
# 6 - repeat 4 & 5 for phase data

In [38]:
#I'll do a version using simple list storage and one with generator/yield to see if there's any benefit to either
def ZygoReadTXT(filename):
    
    with open(filename, "r") as f:
        
        sections = (section for section in f.read().strip().split("#\n") if section.strip() != "")

        headerlines = next(sections).strip().split("\n")
        intensitylines = next(sections).split("\n")
        phaselines = next(sections)
        
        d = (line.split("\"") for line in headerlines)
        stringconstant = next(d)[0]

        #step through each chunk and process as required for the fields we need (compare to Metropro documentation)
        #several fields will need to be converted to int or float types
        #do this by list comprehension on each 'n' in the relevant chunk.split
        chunk = next(d)  # - line 2
        softwareType, majorVersion, minorVersion, bugVers = [int(n) for n in chunk[0].split()]
        softwareDate = chunk[1]

        chunk = next(d)  # - line 3
        intensOriginX, intensOriginY, intensWidth, intensHeight, Nbuckets, intensRange = [int(n) for n in chunk[0].split()]

        chunk = next(d)  # - line 4
        phaseOriginX, phaseOriginY, phaseWidth, phaseHeight = [int(n) for n in chunk[0].split()]

        chunk = next(d)  # - line 5 (comments)
        comments = chunk[1]

        chunk = next(d)  # - line 6
        partSerNum = chunk[1]
        chunk = next(d)  # - line 7
        partNum = chunk[1]

        chunk = next(d)  # - line 8
        source, intfScaleFactor, wavelengthIn, numericAperture, obliquityFactor, magnification, cameraRes, timeStamp = [float(n) for n in chunk[0].split()]

        chunk = next(d)  # - line 9
        cameraWidth, cameraHeight, systemType, systemBoard, systemSerial, instrumentId = chunk[0].split()
        objectiveName = chunk[1]

        chunk = next(d)  # - line 10
        acquireMode, intensAvgs, PZTCal, PZTGain, PZTGainTolerance, AGC, targetRange, lightLevel, minMod, minModPts = chunk[0].split()

        chunk = next(d)  # - line 11
        phaseRes, phaseAvgs, minimumAreaSize, disconAction, disconFilter, connectionOrder, removeTiltBias, dataSign, codeVType = chunk[0].split()

        chunk = next(d)  # - line 12
        subtractSysErr, sysErrFile = int(chunk[0]), chunk[1]

        chunk = next(d)  # - line 13
        refractiveIndex, partThickness = chunk[0].split()

        chunk = next(d)  # - line 14
        zoomDesc = chunk[1]
        
        
        ##################
        #intensity########
        ##################
        intensityrows = []
        for row in intensitylines:
            for val in row.split():
                intensityrows.append(val)

        intensitymap = np.array(intensityrows, dtype=float)
        rows = intensHeight
        cols = intensWidth
        intensitymap = intensitymap.reshape(rows,cols)
    
    return

In [39]:
%timeit ZygoReadTXT("A1.txt")

27.3 ms ± 266 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [1]:
def ZygoReadTXT(filename):
    
    with open(filename, "r") as f:
        sections = [section for section in f.read().strip().split("#\n") if section.strip() != ""]

        headerlines = sections[0].strip().split("\n")
        intensitylines = sections[1].split("\n")
        phaselines = sections[2].split("\n")

        d = [line.split("\"") for line in headerlines]
        stringConstant = d[0][0]

        softwareType, majorVersion, minorVersion, bugVers = [int(n) for n in d[1][0].split()]
        softwareDate = d[1][1]

        intensOriginX, intensOriginY, intensWidth, intensHeight, Nbuckets, intensRange = [int(n) for n in d[2][0].split()]

        # - line 4
        phaseOriginX, phaseOriginY, phaseWidth, phaseHeight = [int(n) for n in d[3][0].split()]

        # - line 5 (comments)
        comments = d[4][1]

        # - line 6
        partSerNum = d[5][1]
        # - line 7
        partNum = d[6][1]

        # - line 8
        source, intfScaleFactor, wavelengthIn, numericAperture, obliquityFactor, magnification, cameraRes, timeStamp = [float(n) for n in d[7][0].split()]

        # - line 9
        cameraWidth, cameraHeight, systemType, systemBoard, systemSerial, instrumentId = d[8][0].split()
        objectiveName = d[8][1]

        # - line 10
        acquireMode, intensAvgs, PZTCal, PZTGain, PZTGainTolerance, AGC, targetRange, lightLevel, minMod, minModPts = d[9][0].split()

        # - line 11
        phaseRes, phaseAvgs, minimumAreaSize, disconAction, disconFilter, connectionOrder, removeTiltBias, dataSign, codeVType = d[10][0].split()

        # - line 12
        subtractSysErr, sysErrFile = int(d[11][0]), d[11][1]

        # - line 13
        refractiveIndex, partThickness = d[12][0].split()

        # - line 14
        zoomDesc = d[13][1]
        
        #####################
        #intensity###########
        #####################
        intensityrows = []
        for row in intensitylines:
            for val in row.split():
                intensityrows.append(val)

        intensitymap = np.array(intensityrows, dtype=float)
        rows = intensHeight - intensOriginY
        cols = intensWidth - intensOriginX
        intensitymap = intensitymap.reshape(rows,cols)
    
    return

In [37]:
%timeit ZygoReadTXT("A1.txt")

27.6 ms ± 104 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [3]:
#list practice - check the types for info being read
g = open("A1.txt", "r")

sections = [section for section in g.read().strip().split("#\n") if section.strip() != ""]

headerlines = sections[0].strip().split("\n")
intensitylines = sections[1].split("\n")
phaselines = sections[2].split("\n")

d = [line.split("\"") for line in headerlines]
stringConstant = d[0][0]

softwareType, majorVersion, minorVersion, bugVers = [int(n) for n in d[1][0].split()]
softwareDate = d[1][1]

intensOriginX, intensOriginY, intensWidth, intensHeight, Nbuckets, intensRange = [int(n) for n in d[2][0].split()]

# - line 4
phaseOriginX, phaseOriginY, phaseWidth, phaseHeight = [int(n) for n in d[3][0].split()]

# - line 5 (comments)
comments = d[4][1]

# - line 6
partSerNum = d[5][1]
# - line 7
partNum = d[6][1]

# - line 8
source, intfScaleFactor, wavelengthIn, numericAperture, obliquityFactor, magnification, cameraRes, timeStamp = [float(n) for n in d[7][0].split()]

# - line 9
cameraWidth, cameraHeight, systemType, systemBoard, systemSerial, instrumentId = d[8][0].split()
objectiveName = d[8][1]

# - line 10
acquireMode, intensAvgs, PZTCal, PZTGain, PZTGainTolerance, AGC, targetRange, lightLevel, minMod, minModPts = d[9][0].split()

# - line 11
phaseRes, phaseAvgs, minimumAreaSize, disconAction, disconFilter, connectionOrder, removeTiltBias, dataSign, codeVType = d[10][0].split()

# - line 12
subtractSysErr, sysErrFile = int(d[11][0]), d[11][1]

# - line 13
refractiveIndex, partThickness = [int(n) for n in d[12][0].split()]

# - line 14
zoomDesc = d[13][1]

In [4]:
refractiveIndex, partThickness

(1, 0)